# Limpieza de datos

Ya familiarizados con nuestro dataset vamos a empezar a limpiar los datos para un futuro análisis

In [4]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../Data/netflix_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Hemos visto en la anterior sección que df.duplicated().sum() == 0, luego no hay filas duplicadas.


Correción de tipo de datos en la columna data_added y verificación de fechas :


In [5]:
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df.info()

# La transformación con pd.to_datetime comprueba fechas imposibles (o NaN) y las convierte en NaT (Not a Time), por lo que ya las tenemoss tratadas.
# Vamos a ver cuántas fechas NaT hay en la columna date_added:
df['date_added'].isna().sum() # --> 98 fechas NaT



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       8807 non-null   object        
 1   type          8807 non-null   object        
 2   title         8807 non-null   object        
 3   director      6173 non-null   object        
 4   cast          7982 non-null   object        
 5   country       7976 non-null   object        
 6   date_added    8709 non-null   datetime64[ns]
 7   release_year  8807 non-null   int64         
 8   rating        8803 non-null   object        
 9   duration      8804 non-null   object        
 10  listed_in     8807 non-null   object        
 11  description   8807 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 825.8+ KB


98

Si el dataframe hubiera estado ordenado por fecha, podríamos haber calculado la fecha media entre dos valores NaT. Pero como no es el caso, y la fecha de publicación tampoco va a cambiar drásticamente el estudio cualitativo del df (pues son 98 de 8807), lo vamos a dejar como NaT.

## Corrección de errores e incoherencias

Sobre todo tenemos que fijarnos en la columna rating, que es la que hemos visto que tiene incoherencias. Para ello, en primer lugar vamos a comprobar si, en las filas donde hay un rating incorrecto (i.e, “64 min”), ese valor coincide con la duración de la película.

In [6]:
df['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)

In [7]:
incorrectos = df['rating'].str.contains('min', na=False) # --> na = False para que los NaN no den error
incorrectos.sum() # --> 3 --> Hay 3 valores incorrectos en la columna 'rating'
df.loc[incorrectos, ['rating', 'duration']]


,rating,duration
5541,74 min,NaN
5794,84 min,NaN
5813,66 min,NaN


La conclusión más obvia que podemos sacar es que hubo un error al registrar los datos y, en estos tres casos, la duración se introdujo en la columna rating. Por ello, vamos a intercambiar los valores en las columnas correspondientes. Además, como no conocemos el rating real de estas películas, en lugar de dejarlo como NaN lo pondremos como “desconocido”.

Obs.: Se trata solo de tres películas, por lo que no sería complicado ni tedioso buscarlas una por una y comprobar su rating. No obstante, nos ponemos en el caso de que fueran n películas a las que les hubiera ocurrido lo mismo, y no vamos a buscar n ratings. Por esa razón, decido marcarlos como “desconocido” en vez de buscar cada uno manualmente.

In [8]:



def mover_valores_incorrectos(df):
    '''
    Esta función mueve los valores incorrectos de la columna 'rating' a la columna 'duration'.
    Los valores incorrectos son aquellos que contienen la cadena 'min'.
    Los valores incorrectos en la columna 'rating' se reemplazan por 'Desconocido'.
    Args:
        df (pd.DataFrame): DataFrame con las columnas 'rating' y 'duration'.
    Returns:
        pd.DataFrame: DataFrame con los valores incorrectos movidos.    

    Tiempo de ejecución: 
        O(n) con n el número de filas del DataFrame.
    '''

    #Iteramos sobre el DataFrame y guardamos los índices de los valores incorrectos en una lista
    list= []
    for i,j in enumerate(df['rating'].str.contains('min', na=False)):
        if j == True:
            list.append(i)
    
    #Obtenemos los valores incorrectos de la columna 'rating' en otra lista para luego copiarlo a la columna 'duration'
    list_valores_incorrectos = df.loc[list, 'rating']

    #Asignamos los valores incorrectos a la columna 'duration'
    for i in list :
        df.loc[i, 'duration'] = list_valores_incorrectos[i]

    #Asignamos el valor 'Desconocido' a los valores incorrectos de la columna 'rating'    
    df.loc[list, 'rating'] = 'Desconocido'

    return df


df = mover_valores_incorrectos(df)
    
df.loc[incorrectos, ['rating', 'duration']]


,rating,duration
5541,Desconocido,74 min
5794,Desconocido,84 min
5813,Desconocido,66 min


Se ha generado una función para realizar el cambio de valores entre ambas columnas. Esta función podría reutilizarse si se trabajase con la misma estructura de dataset en diferentes ocasiones. De hecho, se podría generalizar aún más la función y hacerla de tal manera que sustituyese los valores de dos columnas arbitrarias en las que supiéramos que hay valores intercambiados, sean NaN o de cualquier otro tipo.

In [9]:
def mover_valores_incorrectos(df,colm1,colm2,valor_incorrecto,valor_asignar):

    #Iteramos sobre el DataFrame y guardamos los indices de los valores incorrectos en una lista
    list= []
    for i,j in enumerate(df[colm1].str.contains(valor_incorrecto, na=False)):
        if j == True:
            list.append(i)
    
    #Obtenemos los valores incorrectos de la columna 'colm1' en otra lista para luego copiarlo a la columna 'colm2'
    list_valores_incorrectos = df.loc[list, colm1]


    #Asignamos los valores incorrectos a la columna 'colm2'
    for i in list :
        df.loc[i, colm2] = list_valores_incorrectos[i]

    #Asignamos el valor 'valor_asignar' a los valores incorrectos de la columna 'colm1'    
    df.loc[list, colm1] = valor_asignar

    return df

# la llamada seria : df = mover_valores_incorrectos(df, 'rating', 'duration', 'min', 'Desconocido')

In [10]:
df['release_year'].describe()  # Nada que retocar aquí.

count    8807.000000
mean     2014.180198
std         8.819312
min      1925.000000
25%      2013.000000
50%      2017.000000
75%      2019.000000
max      2021.000000
Name: release_year, dtype: float64

Columna duration

Es cierto que esta columna presenta dos tipos de valores distintos: por un lado, el número de temporadas (en el caso de las series) y, por otro, la duración en minutos de las películas. Se podría intentar generalizar, quizá buscando el tiempo medio en minutos de una temporada y así unificar toda la columna en minutos; sin embargo, puede que los datos estén estructurados así por un motivo.

Por ello, basta con asegurarnos de que todo lo correspondiente a temporadas pertenece a series y que todo lo expresado en minutos corresponde a películas. Con esta verificación, ya tendríamos “limpia” esta columna.

In [11]:
pelis_con_seasons = df[(df["type"] == "Movie") & df["duration"].str.contains("Season", na=False)]
pelis_con_seasons.shape[0] == 0 # --> True, luego no hay peliculas con seasons

series_con_minutes = df[(df["type"] == "TV Show") & df["duration"].str.contains("min", na=False)]
series_con_minutes.shape[0] == 0 # --> True, luego no hay series con minutos



True

## Tratamiento de valores NULOS

In [12]:
df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        98
release_year       0
rating             4
duration           0
listed_in          0
description        0
dtype: int64

En nuestro caso, la mayoría de los nulos están en las columnas director, cast y country. Ninguna de estas columnas permite “inventar” o aproximar valores mediante interpolación lineal, ya que no tendría sentido intentar averiguar el director de una película o el país de origen mediante cálculos numéricos. Por este motivo, se decide sustituir estos valores nulos por "Desconocido".

Obs.: Se podría reducir el número de valores nulos en country con la siguiente técnica: comprobar si el director de esa película ha grabado más películas o series y, en caso afirmativo, si el resto de estas corresponden al mismo país. Si así fuera, podríamos sustituir el dato faltante por el país coincidente. Pero es una técnica un poco rebuscada y aun así no nos asegura el éxito.

In [13]:
df['director'] = df['director'].fillna('Desconocido')
df['cast'] = df['cast'].fillna('Desconocido')
df['country'] = df['country'].fillna('Desconocido')
df['rating'] = df['rating'].fillna('Desconocido')


En el caso de los None de date_added sí vamos a intentar sustituirlos por valores, ya que este valor numérico será de interés a la hora de hacer gráficos y así no perderemos información en ellos. Es la variable determinante para nuestros análisis temporales.

Se decide sustituir el valor de la fecha por la mediana, para obtener un valor representativo que mantenga la consistencia temporal.

In [14]:
df['date_added'] = df['date_added'].fillna(df['date_added'].median())

In [15]:
df.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

Con esto finalizamos nuestro estudio y estimación de valores nulos.
Quedarían dos columnas por analizar: description y listed_in. Estas dos columnas representan una gran variedad de elementos distintos.

In [16]:
len(df.listed_in.unique())  # --> 514 Distintas categorias en listed_in, con varias de una unica aparción 
len(df.description.unique())  # --> 8775 En general las descripciones son únicas, solo hay 32 descripciones repetidas.


8775

En la columna listed_in sí tenemos información que se repite en más de una serie o película. Para un estudio más detallado, se podrían obviar aquellas películas cuya información aparece una única vez y centrarse en todos los casos en los que ya existe repetición.

En general, la agrupación de estos elementos sería recomendable para realizar un análisis más exhaustivo. Una posibilidad sería, al recoger los datos, limitar las opciones posibles para estas categorías; no obstante, ese trabajo no nos corresponde.

Con esto concluimos la parte de limpieza de datos, en la que, a grandes rasgos, se han identificado y solucionado problemas en la columna rating, se han sustituido justificadamente los valores nulos y se ha modificado el tipo de datos de las columnas que presentaban un tipo incoherente.

In [17]:
# Esta última línea de código sirve para guardar el DataFrame limpio en un nuevo archivo CSV
# y poder abrirlo en el notebook Visualizacion.ipynb

df.to_csv("../Data/netflix_clean.csv", index=False)